In [5]:
!pip install tensorflow

In [14]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from sklearn.model_selection import train_test_split
import numpy as np
import matplotlib.pyplot as plt
from glob import glob

# --- Load .npz Data ---
def load_npz_samples(npz_files):
    X_list, Y_list = [], []
    for path in npz_files:
        data = np.load(path)
        X_list.append(data['x'].transpose(1, 2, 0))  # (H, W, 5)
        Y_list.append(data['y'].transpose(1, 2, 0))  # (H, W, 1)
    return np.array(X_list), np.array(Y_list)

all_npz = sorted(glob("training_samples/*.npz"))
X_data, Y_data = load_npz_samples(all_npz)

X_train_val, X_test, Y_train_val, Y_test = train_test_split(X_data, Y_data, test_size=0.1, random_state=42)

X_train, X_val, y_train, y_val = train_test_split(X_data, Y_data, test_size=0.2, random_state=42)

# --- Dice + Combo Loss ---
def dice_loss(y_true, y_pred, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred)
    dice = (2. * intersection + smooth) / (union + smooth)
    return 1 - dice

def combo_loss(y_true, y_pred):
    bce = tf.keras.losses.binary_crossentropy(y_true, y_pred)
    return bce + dice_loss(y_true, y_pred)

# --- IoU Metric ---
def iou_metric(y_true, y_pred):
    y_pred = tf.cast(y_pred > 0.5, tf.float32)
    intersection = tf.reduce_sum(y_true * y_pred)
    union = tf.reduce_sum(y_true) + tf.reduce_sum(y_pred) - intersection
    return intersection / (union + tf.keras.backend.epsilon())

# --- U-Net Model ---
def unet_model(input_shape=(256, 256, 5)):
    inputs = layers.Input(input_shape)

    conv1 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    pool1 = layers.MaxPooling2D((2, 2))(conv1)

    conv2 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(pool1)
    pool2 = layers.MaxPooling2D((2, 2))(conv2)

    conv3 = layers.Conv2D(256, (3, 3), activation='relu', padding='same')(pool2)

    up1 = layers.UpSampling2D((2, 2))(conv3)
    concat1 = layers.Concatenate()([up1, conv2])
    deconv1 = layers.Conv2D(128, (3, 3), activation='relu', padding='same')(concat1)

    up2 = layers.UpSampling2D((2, 2))(deconv1)
    concat2 = layers.Concatenate()([up2, conv1])
    deconv2 = layers.Conv2D(64, (3, 3), activation='relu', padding='same')(concat2)

    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(deconv2)

    return Model(inputs, outputs)

# --- Train Model ---
model = unet_model(input_shape=(256, 256, 5))
model.compile(optimizer='adam', loss=combo_loss, metrics=['accuracy', iou_metric])
model.summary()

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("test_model.keras", save_best_only=True, monitor="val_loss"),
    tf.keras.callbacks.ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=3, verbose=1)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=50,
    batch_size=8,
    callbacks=callbacks
)

# --- Visualize Prediction ---
idx = 0
pred_mask = model.predict(X_val[idx:idx+1])[0] > 0.5

plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.imshow(X_val[idx][:, :, 0], cmap="terrain")
plt.title("Input DEM")

plt.subplot(1, 3, 2)
plt.imshow(y_val[idx].squeeze(), cmap="gray")
plt.title("True Road Mask")

plt.subplot(1, 3, 3)
plt.imshow(pred_mask.squeeze(), cmap="gray")
plt.title("Predicted Road Mask")

plt.tight_layout()
plt.show()

test_dataset = tf.data.Dataset.from_tensor_slices((X_test, Y_test))
test_dataset = test_dataset.batch(8).prefetch(tf.data.AUTOTUNE)

print("Evaluating on test data...")
test_loss, test_accuracy, test_iou = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}, Test Accuracy: {test_accuracy:.4f}, Test IoU: {test_iou:.4f}")


ValueError: With n_samples=0, test_size=0.1 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.